## Preamble

In [1]:
# Import packages

import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import fiona
from tqdm import tqdm

import logging

import atlite

logging.basicConfig(level=logging.INFO)

import io
import pathlib

import numpy as np
import requests
import xarray
from shapely.geometry import Polygon
from rasterio.features import rasterize
from shapely.geometry import mapping
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from rasterio.crs import CRS
from affine import Affine

from rasterio.features import rasterize
from rasterio.transform import from_bounds
from shapely.geometry import box
from zipfile import ZipFile
import numpy as np
import math

from scipy.ndimage import binary_dilation

from rasterio.plot import show
from rasterio.mask import mask

from atlite.gis import ExclusionContainer, shape_availability

## Functions

In [2]:
import os
import fnmatch

def list_strict_tif_files(directory):
    tif_files = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, '*.tif') and not file.endswith('.aux.xml'):
            tif_files.append(os.path.join(directory, file))
    return tif_files

In [3]:
def plot_eligible_area(ax, tiff_path, europe, title):
    excluder_wind_onshore = ExclusionContainer()

    # Ensuring the correct CRS for full_europe
    full_europe = (
        europe
        .assign(col='a')
        .dissolve(by='col')
        .geometry
    )

    # Open the TIFF file and ensure CRS matches
    with rasterio.open(tiff_path) as src:
        if full_europe.crs != src.crs:
            full_europe = full_europe.to_crs(src.crs)

    # Bufffer geometry ensuring CRS consistency
    full_europe = full_europe.to_crs(excluder_wind_onshore.crs)

    # Example mask and transformation logic
    excluder_wind_onshore.add_raster(tiff_path)
    masked, transform = shape_availability(full_europe, excluder_wind_onshore)
    eligible_share = (masked.sum() * excluder_wind_onshore.res**2 / full_europe.geometry.item().area)
    
    # Plot the eligible area in a subplot
    show(masked, transform=transform, cmap='Greens', ax=ax)
    full_europe.plot(ax=ax, edgecolor='k', color='None')
    europe.to_crs(excluder_wind_onshore.crs).boundary.plot(ax=ax, edgecolor='grey', linewidth=0.2)
    ax.set_title(f'{title} Eligible area (green) {eligible_share * 100:2.2f}%')

In [4]:
def buffer_WSF(input_file, output_name, buffer_distance, threshold=1):    
    # Open the existing raster
    with rasterio.open(input_file) as src:
        data = src.read(1, out_dtype=np.uint8)  # Read the first band

        # Binarize the data using the threshold
        binary_data = np.where(data > threshold, 1, 0).astype(np.uint8)
        
        # Use OpenCV to perform dilation with a square structuring element
        structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT, (buffer_distance, buffer_distance))
        buffered_data = cv2.dilate(binary_data, structuring_element)
        
        kwargs = src.meta.copy()
        kwargs.update({
            'dtype': 'uint8',  # Ensure output is 8-bit
            'count': 1,        # Ensure single band
            'crs': CRS.from_epsg(3857),
            'compress': 'lzw'  # Add LZW compression
        })
    
        # Write the buffered raster
        with rasterio.open(output_name, 'w', **kwargs) as dst:
            dst.write(buffered_data, 1)    
            
    #with rasterio.open(output_name, 'w', **kwargs) as dst:
    #    for i in range(1, src.count + 1):
    #        reproject(
    #            source=rasterio.band(src, i),
    #            destination=rasterio.band(dst, i),
    #            src_transform=src.transform,
    #            src_crs=src.crs,
    #            dst_transform=new_transform,
    #            dst_crs=CRS.from_epsg(3857),
    #            resampling=Resampling.nearest)

## Usage

In [5]:
aggregated_regions = [
    "AT", "BE", "BG", "CH", "CZ", "DE",
    "DK", "EE", "ES", "FI", "FR", "UK",
    "GR", "HR", "HU", "IE", "IT", "LT",
    "LU", "LV", "NL", "NO", "PL", "PT", 
    "RO", "SE", "SI", "SK",
]

In [6]:
europe = (
    gpd
    .read_file('/home/oskar/shared_input/geodata/onshore/shapes/NUTS_RG_10M_2021_4326.geojson')
    .query("NUTS_ID == @aggregated_regions")
    .set_index(["NUTS_ID"])
    .loc[:,['geometry']]
)

In [7]:
# The square outer boundaries of Europe to consider, because we have downloaded ERA5 for this extent:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 72

polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)

In [8]:
# The square outer boundaries of Europe to consider, but with CRS ESPG:3857
minx = -1335833.8895192828
maxx = 4898057.594904037
miny = 3895303.9633938945
maxy = 11753184.615338452

In [9]:
import cv2
from concurrent.futures import ProcessPoolExecutor

In [10]:
partial = 'No' # Select No to process all files

directory_path = '/home/oskar/shared_input/geodata/onshore/wimby/WSF/' 
output_dir = '/home/oskar/shared_input/geodata/onshore/wimby/WSF/processed/'
strict_tif_files_list = list_strict_tif_files(directory_path)
set_all = set(strict_tif_files_list)
existing_files = list_strict_tif_files(output_dir)
set_exist = set(existing_files)

buffer_distance = 200
threshold = 1 

needs_processing_set = set_all - set_exist
needs_processing = list(needs_processing_set)

if partial == 'Yes':
    for file in tqdm(needs_processing, desc="Processing Files"):
        output_name = output_dir + file.split('/')[-1]
        buffer_WSF(file, output_name, buffer_distance, threshold)
else: 
    for file in tqdm(strict_tif_files_list, desc="Processing Files"):
        output_name = output_dir + file.split('/')[-1]
        buffer_WSF(file, output_name, buffer_distance, threshold)

Processing Files:   5%|█▏                      | 14/279 [01:23<26:13,  5.94s/it]


KeyboardInterrupt: 

## Merge

In [11]:
from rasterio.vrt import WarpedVRT

In [12]:
def package_warped(files):
    vrt_options = {
        'resampling': rasterio.enums.Resampling.nearest,
        'nodata': None,
        'add_alpha': False
    }

    for fp in files:
        with rasterio.open(fp) as src:
            vrt = WarpedVRT(src, **vrt_options)
            yield vrt


In [16]:
processed_files = list_strict_tif_files(output_dir)
src_files_to_mosaic = list(package_warped(processed_files))

# Merge the raster files
mosaic, out_transform = merge(src_files_to_mosaic)

# Define the output file path
output_file = '/home/oskar/shared_input/geodata/onshore/wimby/processed/WSF_merged.tif'

# Copy the metadata from one of the original files
out_meta = src_files_to_mosaic[0].meta.copy()

# Update the metadata to match the merged data
out_meta.update({
    'driver': 'GTiff',
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'transform': out_transform,
    'count': mosaic.shape[0],
    'compress': 'lzw'
})

MemoryError: Unable to allocate 175. GiB for an array with shape (1, 400974, 467765) and data type uint8

In [13]:
import tempfile

In [14]:
def merge_files_incrementally(file_list, out_file_path, temp_dir):
    temp_files = []

    for i, file in tqdm(enumerate(file_list)):
        with rasterio.open(file) as src:
            temp_path = os.path.join(temp_dir, f"temp_merged_{i}.tif")
            
            # Include compression in the temporary file metadata
            temp_meta = src.meta.copy()
            temp_meta.update({
                'compress': 'lzw'
            })
            
            with rasterio.open(temp_path, "w", **temp_meta) as dest:
                dest.write(src.read())
                
            temp_files.append(temp_path)

    # Process merge again in chunks
    open_files = [rasterio.open(f) for f in temp_files]
    try:
        mosaic, out_transform = merge(open_files)
        out_meta = open_files[0].meta.copy()
        out_meta.update({
            'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': out_transform,
            'count': mosaic.shape[0],
            'compress': 'lzw'
        })
        with rasterio.open(out_file_path, 'w', **out_meta) as dest:
            dest.write(mosaic)
    finally:
        for f in open_files:
            f.close()
    
    # Cleanup temp files
    for temp_file in temp_files:
        os.remove(temp_file)

In [ ]:
def merge_files_chunked(file_list, out_file_path, temp_dir):
    temp_files = []
    
    for i, file in tqdm(enumerate(file_list, 1), desc="Preparing temporary files"):
        with rasterio.open(file) as src:
            temp_path = os.path.join(temp_dir, f"temp_{i}.tif")
            temp_meta = src.meta.copy()
            temp_meta.update({'compress': 'lzw'})
            
            with rasterio.open(temp_path, "w", **temp_meta) as temp_dst:
                temp_dst.write(src.read())
                
            temp_files.append(temp_path)

    # Create a virtual raster (VRT) to handle large data efficiently
    open_files = [rasterio.open(f) for f in temp_files]
    try:
        mosaic, out_transform = merge(open_files, use_memory_frac=0.1)  # Use only a fraction of available memory
        out_meta = open_files[0].meta.copy()
        out_meta.update({
            'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': out_transform,
            'count': mosaic.shape[0],
            'compress': 'lzw',
            'dtype': 'uint8'
        })
        with rasterio.open(out_file_path, 'w', **out_meta) as dest:
            dest.write(mosaic)
    finally:
        for f in open_files:
            f.close()
    
    # Clean up temporary files
    for temp_file in temp_files:
        os.remove(temp_file)

In [15]:
processed_files = list_strict_tif_files(output_dir)
temp_directory = tempfile.mkdtemp()
output_file = '/home/oskar/shared_input/geodata/onshore/wimby/processed/WSF_merged_test.tif'
merge_files_incrementally(processed_files, output_file, temp_directory)

15it [00:27,  1.85s/it]


MemoryError: Unable to allocate 175. GiB for an array with shape (1, 400974, 467765) and data type uint8

## Plots

In [ ]:
name_dict = {
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/airport_airports_buffered.tif' : 'Airports 5km buffer',
    '/home/oskar/shared_input/geodata/onshore/euro_slope_40degs.tif' : 'Slope 36°',
    '/home/oskar/shared_input/geodata/onshore/wimby//processed/15degrees.tif' : 'Slope 15°',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/powerlines_powerlines_buffered.tif' : 'Powerlines',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/rail_narrow_buffered.tif' : 'Rail - Narrow',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/rail_rail_buffered.tif' : 'Rail - Rail',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/glacier_glacier_buffered.tif' : 'Glaciers',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_motorway_link_buffered.tif' : 'Roads - Motorways_link',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_motorway_buffered.tif' : 'Roads - Motorways_motorway',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_trunk_buffered.tif' : 'Roads - Motorways_trunk',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_primary_primary_buffered.tif': 'Roads - Primary',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/military_military_buffered.tif': 'Military 5.5km buffer',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/radar_radar_buffered.tif': 'Radars 6km buffer',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/2000m.tif': 'Elevation (2000m)',
}

In [ ]:
tiff_paths = [
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/airport_airports_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/15degrees.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/powerlines_powerlines_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/rail_narrow_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/rail_rail_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_motorway_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_trunk_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/glacier_glacier_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_motorways_motorway_link_buffered.tif',
    #'/home/oskar/shared_input/geodata/onshore/wimby/processed/military_military_buffered.tif',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/radar_radar_buffered.tif',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/2000m.tif',
    '/home/oskar/shared_input/geodata/onshore/wimby/processed/roads_primary_primary_buffered.tif',
]

In [ ]:
# Create subplots - one for each file
n_files = len(tiff_paths)
columns = 3
rows = math.ceil(n_files/columns)

fig, axes = plt.subplots(rows, columns, figsize=(5 * columns, 5 * rows))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each file on the corresponding subplot
for ax, tiff_path in tqdm(zip(axes, tiff_paths)):
    print(f'Plotting {tiff_path}')
    plot_eligible_area(ax, tiff_path, europe, f'{name_dict.get(tiff_path)}: ')

# Turn off unused subplots
for ax in axes[n_files:]:
    ax.set_visible(False)

plt.tight_layout()
plt.savefig('Individual exclusions.pdf',bbox_inches='tight')
plt.show()
